In [2]:
import pandas as pd 

In [3]:
df = pd.read_csv('dataset.csv')

species_counts = df["species"].value_counts()
print(species_counts.head(3))

species
Caenorhabditis elegans     1559
Drosophila melanogaster     926
Mus musculus                277
Name: count, dtype: int64


Let's isolate our dataset to only rows that with the Canorhabiditis elegans species, since it has the most data points 

In [4]:
df = df[df["species"] == "Canorhabiditis elegans"]

We can now condense multiples rows that belong to the same strain into singular rows that represent summary statistics of that strain 